![logo](https://neutouch.eu/templates/yootheme/cache/g37-01577415.png)
# NeuTouch summer school on Touch and Robotics
Tutorial: Slip Detection with Neural Networks

In this tutorial we will look into incipient slip detection and slip classification with deep neural networks. We will experiment with and evaluate various neural network architectures to detect and classify slippage.
To this end, we will work on a pre-recorded data set from our 16x16 tactile sensor array, comprising tactile time series data for three different situations:
- stable grasp condition
- translational slip
- rotational slip


In [ ]:
# Let's download the data set first:
!wget "https://uni-bielefeld.sciebo.de/s/7vi5lX9WO1VmvIZ/download" -O "data_stable_slip_rotate.pkl"

In [ ]:
# Actually load the data
import pickle

data = pickle.load(open("data_stable_slip_rotate.pkl", "rb"), encoding='latin1')

## Task: Visually inspect the data

Use [matplotlib's animation](https://matplotlib.org/stable/api/animation_api.html) to create videos for all three classes (stable, translation, rotation)
and [embed them into the notebook](http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-as-interactive-javascript-widgets/).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# The data is organized into three classes:
fig = plt.figure()  # create a new figure
for idx, key in enumerate(data.keys()):
    ax = fig.add_subplot(1, 3, idx+1, xticks=[], yticks=[],  # add subplot w/o any axis ticks
                         xlabel="{name}: {shape}".format(name=key, shape=data[key].shape))
    ax.imshow(data[key][np.random.randint(0, len(data[key]))], cmap = "Greys")  # plot a random sample

In [ ]:
import random
import matplotlib.animation as animation
from IPython.display import HTML

# Limit video to a subset of the data
subset = data['translation'][::100]

# Set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize=(2, 2))
im = plt.imshow(subset[0], cmap='Greys')


def animate_func(i):
    im.set_array(subset[i])
    return [im]


anim = animation.FuncAnimation(fig, animate_func,
                               frames=len(subset),
                               interval=50,  # in ms
                               )
plt.close(fig)
HTML(anim.to_html5_video())

## Baseline classification of raw data

As a baseline approach, we illustrate here how to use a CNN to classify the raw data.
As seen above, each class comprises 50k frames of consecutive sensor recordings, recorded at a frame rate of 1kHz. Hence, we have 50s of data for each class.
Obviously, the network cannot predict from a single sample whether there is slippage or not. So, let's chop the whole time-series into short sequences of fixed length, say 32 samples, which then can be fed into a neural network.

In [ ]:
# Import Keras + TensorFlow packages
import tensorflow as tf
from tensorflow import keras as keras

# https://colab.research.google.com/notebooks/gpu.ipynb
print("TensorFlow version:", tf.__version__)
print("GPU:", tf.test.gpu_device_name())

# Define a random seed to have deterministic results
np.random.seed(11)

In [ ]:
# Split data set into train and test
def train_test_split(X, Y, ratio=0.8):
   split=int(len(X)*ratio)
   X_train = X[:split]
   Y_train = Y[:split]

   X_test = X[split:]
   Y_test = Y[split:]

   return X_train, X_test, Y_train, Y_test

In [ ]:
# Split long time series into chunks of given length and stride
def chop_time_series(series, length=32, stride=8):
   # https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/TimeseriesGenerator
   N = len(series)
   chunks, _ = keras.preprocessing.sequence.TimeseriesGenerator(
      series, np.zeros(N), length=length, stride=stride, batch_size=N)[0]
   # TimeseriesGenerator yields a format like (N, width, height)
   # However TensorFlow uses channels_last as its default "image" format, i.e. (width, height, N)
   # Thus, transpose into correct format:
   return chunks.transpose(0, 2, 3, 1)

In [ ]:
# Prepare the dataset for training: chop into sequences of given length and assign numeric class labels
def prepare_dataset(length=32, stride=8, ratio=0.8):
   # map textual labels onto numeric class labels
   label_mapping = dict(stable=0, translation=1, rotation=2)
   x_train_test_y_train_test = []
   for key in data.keys():
      x = chop_time_series(data[key], length, stride)
      y = np.full(len(x), label_mapping[key])  # generate numeric label for all xs
      # optionally perform data augmentation (same on all elements of chunk!)
      # optionally perform FFT on input data X
      # https://numpy.org/doc/stable/reference/generated/numpy.fft.rfft.html
      print(key, x.shape)
      # normalize 12bit ADC data (0..2^12) into range (0..1)
      x_train_test_y_train_test.append(train_test_split(x / 4096, y, ratio=ratio))

   # combine data from all classes into a single data set
   return (np.concatenate([x_train_test_y_train_test[cls][sub] for cls in range(len(x_train_test_y_train_test))]) for sub in range(4))


X_train, X_test, Y_train, Y_test = prepare_dataset()

In [ ]:
# Simple MLP with a single hidden layer
def simple_mlp(num_classes=3):
   model = keras.models.Sequential(name="SimpleMLP")
   model.add(keras.layers.Flatten())  # Flatten input tensor into single vector
   model.add(keras.layers.Dense(64, activation='relu'))  # hidden layer with 64 neurons
   model.add(keras.layers.Dense(num_classes))  # dense classification layer
   return model

# Basic network with a single CNN layer
def simple_cnn(num_classes=3):
   model = keras.models.Sequential(name="SimpleCNN")
   model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
   model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
   model.add(keras.layers.Flatten())  # Flatten tensor into single vector
   model.add(keras.layers.Dense(num_classes))  # dense classification layer
   return model


In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf logs/

In [ ]:
def train(model):
   model.build(input_shape=X_train.shape)
   model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer=keras.optimizers.RMSprop(),
                 metrics=['sparse_categorical_accuracy'])
   model.summary()

   # Specify log directory for TensorBoard
   log_dir = "logs/" + model.name

   # Initialize TensorBoard
   tb = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

   # Fit data to the model!
   model.fit(X_train, Y_train,
             batch_size=16, epochs=10,
             validation_data=(X_test, Y_test),
             callbacks=[tb])

   score = model.evaluate(X_test, Y_test, verbose=False)
   print('Test loss:', score[0])
   print('Test accuracy:', score[1])
   return model

In [ ]:
mlp = train(simple_mlp())
cnn = train(simple_cnn())

In [ ]:
# Inspect results with TensorBoard
%tensorboard --logdir logs

## Task: Sanity check

- The perfect classification accuracy is suspecious! Why is it so trivial for the networks to classify the data? Have a look at your data visualizations and compare the videos of the 3 classes!
- To increase the variability of the data set, use [data augmentation techniques](https://www.tensorflow.org/tutorials/images/data_augmentation), e.g. to rotate the input data by 90°, 180°, 270° and shift by a few taxels.
- As pointed out in the lecture, we are looking for micro vibrations. Thus introduce [FFT](https://numpy.org/doc/stable/reference/generated/numpy.fft.rfft.html) preprocessing. Are frequencies and/or phases important?
- Instead of considering the whole 16x16 array as input, we could split the array into smaller units, e.g. 2x2 patches. This will make the problem harder, because the network needs to recognize slippage from those smaller patches. On the other hand, we also increase the dataset. Ensure that you only include patches that actually have contact!

## Task: Experiment!
Try various networks:
- Add more CNN layers
- Adapt size of layers, size of kernels, num of kernels
- Experiment with different optimizers: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers

Improve generalization
- Introduce drop-out: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout
- Use batch normalization: https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization